In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/feedback-prize-english-language-learning/sample_submission.csv
/kaggle/input/feedback-prize-english-language-learning/train.csv
/kaggle/input/feedback-prize-english-language-learning/test.csv
/kaggle/input/iterativestratification/.travis.yml
/kaggle/input/iterativestratification/setup.cfg
/kaggle/input/iterativestratification/LICENSE
/kaggle/input/iterativestratification/.gitignore
/kaggle/input/iterativestratification/README.md
/kaggle/input/iterativestratification/setup.py
/kaggle/input/iterativestratification/tests/test_ml_stratifiers.py
/kaggle/input/iterativestratification/tests/__init__.py
/kaggle/input/iterativestratification/iterstrat/ml_stratifiers.py
/kaggle/input/iterativestratification/iterstrat/__init__.py
/kaggle/input/roberta-base/rust_model.ot
/kaggle/input/roberta-base/config.json
/kaggle/input/roberta-base/merges.txt
/kaggle/input/roberta-base/README.md
/kaggle/input/roberta-base/tokenizer.json
/kaggle/input/roberta-base/vocab.json
/kaggle/input/roberta-

# Load libraries

In [2]:
import os, gc
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras import layers

import transformers
from transformers import TFAutoModel, AutoTokenizer, AutoConfig
import sys 
sys.path.append('../input/iterativestratification') 
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold


# Import Dataset

In [3]:
# import datasets
train = pd.read_csv('../input/feedback-prize-english-language-learning/train.csv')
test = pd.read_csv('../input/feedback-prize-english-language-learning/test.csv')
sample_submission = pd.read_csv('../input/feedback-prize-english-language-learning/sample_submission.csv')

train.head(), test.head(), sample_submission.head()

(        text_id                                          full_text  cohesion  \
 0  0016926B079C  I think that students would benefit from learn...       3.5   
 1  0022683E9EA5  When a problem is a change you have to let it ...       2.5   
 2  00299B378633  Dear, Principal\n\nIf u change the school poli...       3.0   
 3  003885A45F42  The best time in life is when you become yours...       4.5   
 4  0049B1DF5CCC  Small act of kindness can impact in other peop...       2.5   
 
    syntax  vocabulary  phraseology  grammar  conventions  
 0     3.5         3.0          3.0      4.0          3.0  
 1     2.5         3.0          2.0      2.0          2.5  
 2     3.5         3.0          3.0      3.0          2.5  
 3     4.5         4.5          4.5      4.0          5.0  
 4     3.0         3.0          3.0      2.5          2.5  ,
         text_id                                          full_text
 0  0000C359D63E  when a person has no experience on a job their...
 1  000BAD50D02

# Create Cross-Validation split

In [4]:
# create 5 folds for cross validation
N_FOLD = 5
TARGET_COLS = ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']

In [5]:
# apply kfold on dataset
folds = MultilabelStratifiedKFold(n_splits=N_FOLD, shuffle=True, random_state=42)

for n, (train_index, val_index) in enumerate(folds.split(train, train[TARGET_COLS])):
    train.loc[val_index, 'fold'] = int(n)

train['fold'] = train['fold'].astype(int)
train['fold'].value_counts()

1    783
0    782
4    782
3    782
2    782
Name: fold, dtype: int64

In [6]:
# check train dataset after adding fold column
train.head()

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions,fold
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0,1
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5,0
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5,4
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0,3
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5,1


# Model configuration

In [7]:
# standard config
MAX_LENGTH = 512
BATCH_SIZE = 8

MODEL = "../input/roberta-base/"

In [8]:
# load Roberta model 

model = TFAutoModel.from_pretrained(MODEL)
tokenizer = AutoTokenizer.from_pretrained(MODEL)

CFG = transformers.AutoConfig.from_pretrained(MODEL)
CFG.hidden_dropout_prob = 0.
CFG.attention_probs_dropout_prob = 0.

2022-11-01 10:44:14.048519: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-01 10:44:14.049577: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-01 10:44:14.050262: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-01 10:44:14.052135: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

# Data Processing Functions

In [9]:
def CreateEmbeddings(texts, tokenizer=tokenizer):
    
    """Generate embeddings and attention_masks from text """

    input_ids = []
    attention_mask = []
    
    for text in texts.to_list():
        token = tokenizer(text, 
                          add_special_tokens=True,
                          max_length=MAX_LENGTH,
                          return_attention_mask=True, 
                          return_tensors='np',
                          truncation=True,
                          padding='max_length')
        input_ids.append(token['input_ids'][0])
        attention_mask.append(token['attention_mask'][0])
    
    return np.array(input_ids), np.array(attention_mask) 

In [10]:
def GetDataset(dataframe):
    
    """Create dataset object from dataframe"""
    
    inputs = CreateEmbeddings(dataframe['full_text'])
    targets = np.array(dataframe[TARGET_COLS])
    
    return inputs, targets

# MeanPooling Layer

Instead of using '[CLS]' token, MeanPool layer averaging roberta's last hidden states along the sequence axis with masking out padding tokens.

In [11]:
class MeanPooling(tf.keras.layers.Layer):
    
    def call(self, inputs, mask=None):
        
        """Generate sentence embeddings for each text by applying mean pooling """
        
        total_mask = tf.expand_dims(tf.cast(mask, "float32"), -1)
        embedding_sum = tf.reduce_sum(inputs * total_mask, axis=1)
        mask_sum = tf.reduce_sum(total_mask, axis=1)
        mask_sum = tf.math.maximum(mask_sum, tf.constant([1e-9]))
        
        return embedding_sum / mask_sum

# Build GradientBoosting Model

In [13]:
import tqdm

In [16]:
# get the last hidden state layer from tokens
embeddings = []

for text in train['full_text'].to_list():
#     inputs = tokenizer(text, padding=True, truncation=True, return_tensors="np", max_length=512)
    
    inputs = tokenizer(text, 
                  add_special_tokens=True,
                  max_length=MAX_LENGTH,
                  return_attention_mask=True, 
                  return_tensors='np',
                  truncation=True,
                  padding='max_length')
    
    outputs = model(inputs)
    embeddings.extend(outputs.last_hidden_state)

In [ ]:
EMB_use = []    

for i in range(len(embeddings)):
    
    a = embeddings[i]
    mean_pool = []
    
    for j in range(len(a)):
        
        b = a[j]
        n = np.mean(b)
        mean_pool.append(n)

    EMB_use.append(mean_pool)

In [177]:
# create mean pooling embeddings 
test_embeddings = []

for text in test['full_text'].to_list():
    
    inputs = tokenizer(text, 
                  add_special_tokens=True,
                  max_length=MAX_LENGTH,
                  return_attention_mask=True, 
                  return_tensors='np',
                  truncation=True,
                  padding='max_length')
    
    outputs = model(inputs)
    test_embeddings.extend(outputs.last_hidden_state)
    
EMB_test = []    

for i in range(len(test_embeddings)):
    
    a = test_embeddings[i]
    mean_pool = []
    
    for j in range(len(a)):
        
        b = a[j]
        n = np.mean(b)
        mean_pool.append(n)

    EMB_test.append(mean_pool)

# 5 Folds training loop

In [86]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LinearRegression,SGDRegressor
from scipy import sparse
from sklearn.multioutput import MultiOutputRegressor
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
preds = []
scores = []

In [159]:
def comp_score(y_true,y_pred):
    """ create evaluation function """
    rmse_scores = []
    for i in range(len(TARGET_COLS)):
        rmse_scores.append(np.sqrt(mean_squared_error(y_true[:,i],y_pred[:,i])))
    return np.mean(rmse_scores)

In [253]:
# input embeddings into GradientBoosting and start training. monitoring best fold performance

preds = []
scores = []

EMB_use = np.array(EMB_use)
EMB_test = np.array(EMB_test)

for fold in range(N_FOLD):
    
    train_df = train[train['fold'] != fold]
    val_df = train[train['fold'] == fold]
    
    train_emb = EMB_use[train_df.index]
    val_emb = EMB_use[val_df.index]
    test_emb = EMB_test
    
    clf = MultiOutputRegressor(GradientBoostingRegressor())
    
    clf.fit(train_emb, train_df[TARGET_COLS].values)
    
    preds = clf.predict(val_emb)
    
    score = comp_score(val_df[TARGET_COLS].values,preds)
    
    scores.append(score)
    
    print("Fold : {} EV score: {}".format(fold,score))
    
    test_preds = list(clf.predict(test_emb))
    
    preds = (preds, test_preds)
#     preds.concanate(test_preds)
    
print(np.mean(scores))

Fold : 0 EV score: 0.5499660397302873
Fold : 1 EV score: 0.5644912308718081
Fold : 2 EV score: 0.5683711627431417
Fold : 3 EV score: 0.5658749510864204
Fold : 4 EV score: 0.5608917586839836
0.5619190286231281


# Create predictions for test

In [216]:
# create submission file
submission = test.copy()

submission

,text_id,full_text
0,0000C359D63E,when a person has no experience on a job their...
1,000BAD50D026,Do you think students would benefit from being...
2,00367BB2546B,"Thomas Jefferson once states that ""it is wonde..."


In [274]:
submission.loc[:, TARGET_COLS] = preds[1]

submission = submission.drop(columns='full_text', inplace=True)

In [278]:
submission.to_csv('submission.csv', index=False)

submission.head()

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0000C359D63E,when a person has no experience on a job their...,3.281994,3.122031,3.227372,3.153807,2.955520,3.108195
1,000BAD50D026,Do you think students would benefit from being...,2.998643,2.724139,3.113894,2.883073,2.997155,3.167511
2,00367BB2546B,"Thomas Jefferson once states that ""it is wonde...",3.345882,3.240552,3.446494,3.447341,3.263307,3.212591


In [276]:
# 